# 📊 Guía de Usuario: mcp-bcrp

Bienvenido a la guía oficial de **mcp-bcrp**, la librería y servidor MCP para acceder a las estadísticas del **Banco Central de Reserva del Perú (BCRP)** de manera profesional y eficiente.

[![GitHub](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/MaykolMedrano/mcp_bcrp)
[![PyPI](https://img.shields.io/pypi/v/mcp-bcrp.svg)](https://pypi.org/project/mcp-bcrp/)

Esta guía te llevará desde la instalación básica hasta la generación de gráficos avanzados y análisis de datos.

## 1. Instalación

Primero, instalamos la librería desde PyPI. Recomendamos incluir `[charts]` para poder generar visualizaciones.

In [ ]:
!pip install mcp-bcrp[charts] -U

# NOTA: En Google Colab, es posible que necesites reiniciar el entorno (Runtime -> Restart session)
# después de la instalación para que los cambios surtan efecto.

## 2. Configuración y Metadatos

La librería utiliza un motor de búsqueda determinista que requiere un catálogo de metadatos (~17MB). La primera vez que lo uses, se descargará automáticamente y se guardará en tu cache local.

El cache se guarda en:
- Windows: `%LOCALAPPDATA%\mcp_bcrp\bcrp_metadata.json`
- Linux/Mac/Colab: `~/.cache/mcp_bcrp/bcrp_metadata.json`

In [ ]:
import asyncio
from mcp_bcrp.client import AsyncBCRPClient, BCRPMetadata
import pandas as pd

# Inicializamos el cliente de metadatos
metadata = BCRPMetadata()

# Cargamos los metadatos (descarga automática si no existe)
# En Notebooks modernos, puedes usar 'await' directamente en la celda.
await metadata.load()

print(f"Metadatos cargados: {len(metadata.df)} series disponibles.")

## 3. Búsqueda de Indicadores

### Búsqueda Determinista
El motor de búsqueda es **determinista**. Puede encontrar series específicas basándose en atributos extraídos de tu texto.

In [ ]:
# Ejemplo: Buscar la tasa de interés de política monetaria
query = "tasa interes politica monetaria"
resultado = metadata.solve(query)

print("Resultado de búsqueda determinista:")
print(f"Código: {resultado['codigo_serie']}")
print(f"Nombre: {resultado['name_original']}")
print(f"Confianza: {resultado['confidence']}")

### Búsqueda Exploratoria
Si no estás seguro de lo que buscas, puedes explorar el catálogo usando búsqueda difusa.

In [ ]:
# Explorar indicadores relacionados con 'Inflación'
exploracion = metadata.search("inflacion", limit=5)
display(exploracion[["Código de serie", "Nombre de serie"]])

## 4. Obtención de Datos (Series Temporales)

Usamos `AsyncBCRPClient` para descargar los datos. El cliente detecta automáticamente la frecuencia (diaria, mensual, trimestral, anual).

In [ ]:
client = AsyncBCRPClient()

# Descargamos datos: Inflación (Variación IPC) y Tipo de Cambio Venta (Diario)
series = ["PN01271PM", "PD04637PD"]

df = await client.get_series(
    codes=series,
    start_date="2023-01-01",
    end_date="2024-12-31"
)

print("Primeras filas de los datos descargados:")
display(df.head())

## 5. Visualización Profesional

La librería centraliza la lógica de visualización para que no tengas que preocuparte por el formato de fechas del BCRP.

In [ ]:
from mcp_bcrp.server import _plot_chart

# Generamos un gráfico comparativo de Expectativas de Crecimiento
img_path = "comparativo_expectativas.png"

await _plot_chart(
    series_codes=["PD38048AM", "PD38049AM"], # Expectativas PBI 12 meses y 3 meses
    period="2018-01/2024-12",
    title="Expectativas de Crecimiento del PBI",
    names=["Exp. 12 meses", "Exp. 3 meses"],
    output_path=img_path
)

from IPython.display import Image
Image(img_path)

## 6. Análisis Estadístico

Con `pandas`, el análisis está a un paso de distancia.

In [ ]:
resumen = df.describe().T
print("Resumen estadístico:")
display(resumen)

---
### Recursos Adicionales

- **GitHub**: [MaykolMedrano/mcp_bcrp](https://github.com/MaykolMedrano/mcp_bcrp)
- **PyPI**: [mcp-bcrp](https://pypi.org/project/mcp-bcrp/)

¡Gracias por usar **mcp-bcrp**! Si encuentras algún error o tienes sugerencias, no dudes en abrir un Issue en GitHub.